<a href="https://colab.research.google.com/github/miingaluque/hello-world/blob/main/Pyspark_Colab_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark en Google Colab Automatico


## Instalacion de Java
Google Colaboratory funciona en un ambiente linux, por lo tanto se pueden usar comandos shell de linux antecedidos del caracter '!'

In [ ]:
# verificar si ya esta isntalado en "/usr/lib/jvm/" , si esta instalado omitir esta linea
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null

## Instalacion de Spark

Obtener automaticamente la ultima version de spark de 

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
#Obtener las versiones de spark la pagina web
url = 'https://downloads.apache.org/spark/' 
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)

In [ ]:
# leer la pagina web y obtener las versiones de spark disponibles
link_files = []
for link in soup.find_all('a'):
  link_files.append(link.get('href'))
spark_link = [x for x in link_files if 'spark' in x]  
print(spark_link)

['spark-2.4.8/', 'spark-3.0.3/', 'spark-3.1.2/', 'spark-3.2.1/']


La version a usar seran las superiores a spark-2.3.4  y menores a spark-3.0.0

obtener la version y eliminar el caracter '/' del final

In [ ]:
ver_spark = spark_link[1][:-1] # obtener la version y eliminar el caracter '/' del final
print(ver_spark)

spark-3.0.3


In [ ]:
import os # libreria de manejo del sistema operativo
#instalar automaticamente la version deseadda de spark
# Verificar la ruta de descarga por actualizaciones
os.system(f"wget -q https://downloads.apache.org/spark/{ver_spark}/{ver_spark}-bin-hadoop2.7.tgz")
os.system(f"tar xf {ver_spark}-bin-hadoop2.7.tgz")
# instalar pyspark
!pip install -q pyspark

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 52.5 MB/s 


## Definir variables de entorno

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{ver_spark}-bin-hadoop2.7"

In [ ]:
!pip install findspark
import findspark
findspark.init() 

# Cargar pyspark en el sistema

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()
spark

# Ejemplo de Uso de pyspark

Leer archivo de prueba

In [ ]:
CV_data = spark.read.load('churn-bigml-80.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [ ]:
CV_data.cache()
CV_data.printSchema()

root
 |-- State: string (nullable = true)
 |-- Account length: integer (nullable = true)
 |-- Area code: integer (nullable = true)
 |-- International plan: string (nullable = true)
 |-- Voice mail plan: string (nullable = true)
 |-- Number vmail messages: integer (nullable = true)
 |-- Total day minutes: double (nullable = true)
 |-- Total day calls: integer (nullable = true)
 |-- Total day charge: double (nullable = true)
 |-- Total eve minutes: double (nullable = true)
 |-- Total eve calls: integer (nullable = true)
 |-- Total eve charge: double (nullable = true)
 |-- Total night minutes: double (nullable = true)
 |-- Total night calls: integer (nullable = true)
 |-- Total night charge: double (nullable = true)
 |-- Total intl minutes: double (nullable = true)
 |-- Total intl calls: integer (nullable = true)
 |-- Total intl charge: double (nullable = true)
 |-- Customer service calls: integer (nullable = true)
 |-- Churn: boolean (nullable = true)



In [ ]:
# Numero de observaciones
CV_data.count()

2666

In [ ]:
# La data fue correctamenta cargada
CV_data.take(5)

[Row(State='KS', Account length=128, Area code=415, International plan='No', Voice mail plan='Yes', Number vmail messages=25, Total day minutes=265.1, Total day calls=110, Total day charge=45.07, Total eve minutes=197.4, Total eve calls=99, Total eve charge=16.78, Total night minutes=244.7, Total night calls=91, Total night charge=11.01, Total intl minutes=10.0, Total intl calls=3, Total intl charge=2.7, Customer service calls=1, Churn=False),
 Row(State='OH', Account length=107, Area code=415, International plan='No', Voice mail plan='Yes', Number vmail messages=26, Total day minutes=161.6, Total day calls=123, Total day charge=27.47, Total eve minutes=195.5, Total eve calls=103, Total eve charge=16.62, Total night minutes=254.4, Total night calls=103, Total night charge=11.45, Total intl minutes=13.7, Total intl calls=3, Total intl charge=3.7, Customer service calls=1, Churn=False),
 Row(State='NJ', Account length=137, Area code=415, International plan='No', Voice mail plan='No', Num

In [ ]:
# Usamos pandas para mostrar los datos cargados con spark
import pandas as pd
pd.DataFrame(CV_data.take(50)[15:25], columns=CV_data.columns)

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,VA,76,510,No,Yes,33,189.7,66,32.25,212.8,65,18.09,165.7,108,7.46,10.0,5,2.70,1,False
1,TX,73,415,No,No,0,224.4,90,38.15,159.5,88,13.56,192.8,74,8.68,13.0,2,3.51,1,False
2,FL,147,415,No,No,0,155.1,117,26.37,239.7,93,20.37,208.8,133,9.40,10.6,4,2.86,0,False
3,CO,77,408,No,No,0,62.4,89,10.61,169.9,121,14.44,209.6,64,9.43,5.7,6,1.54,5,True
4,AZ,130,415,No,No,0,183.0,112,31.11,72.9,99,6.20,181.8,78,8.18,9.5,19,2.57,0,False
5,VA,132,510,No,No,0,81.1,86,13.79,245.2,72,20.84,237.0,115,10.67,10.3,2,2.78,0,False
6,NE,174,415,No,No,0,124.3,76,21.13,277.1,112,23.55,250.7,115,11.28,15.5,5,4.19,3,False
7,WY,57,408,No,Yes,39,213.0,115,36.21,191.1,112,16.24,182.7,115,8.22,9.5,3,2.57,0,False
8,MT,54,408,No,No,0,134.3,73,22.83,155.5,100,13.22,102.1,68,4.59,14.7,4,3.97,3,False
9,MO,20,415,No,No,0,190.0,109,32.30,258.2,84,21.95,181.5,102,8.17,6.3,6,1.70,0,False


## Machine learning

In [ ]:
# Removemos the State , Area code , Total day charge , Total eve charge , Total night charge and Total intl charge
CV_data = CV_data.drop('State').drop('Area code') \
    .drop('Total day charge').drop('Total eve charge') \
    .drop('Total night charge').drop('Total intl charge')

In [ ]:
#Transforms 'Yes' and 'No' to '1.0' or '0.0'
def transform1(value):
    result=-1.0
    if value=='Yes':
        result=1.0
    if value=='No':
        result=0.0
    return result

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

transform1_udf = udf(lambda k: transform1(k), DoubleType())

In [ ]:
# Se aplica withcolumn
CV_data = CV_data.withColumn('International plan', transform1_udf(CV_data['International plan'])) \
    .withColumn('Voice mail plan', transform1_udf(CV_data['Voice mail plan'])).cache()

In [ ]:
#Transforms 'Yes' and 'No' to '1.0' or '0.0'
def transform2(value):
    dic={False: 0.0, True:1.0}
    return dic[value]

transform2_udf= udf(lambda k: transform2(k), DoubleType())

CV_data=CV_data.withColumn('Churn', transform2_udf(CV_data['Churn'])).cache()

In [ ]:
# Muestro los cambios
pd.DataFrame(CV_data.take(5), columns=CV_data.columns)

,Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total eve minutes,Total eve calls,Total night minutes,Total night calls,Total intl minutes,Total intl calls,Customer service calls,Churn
0,128,0.0,1.0,25,265.1,110,197.4,99,244.7,91,10.0,3,1,0.0
1,107,0.0,1.0,26,161.6,123,195.5,103,254.4,103,13.7,3,1,0.0
2,137,0.0,0.0,0,243.4,114,121.2,110,162.6,104,12.2,5,0,0.0
3,84,1.0,0.0,0,299.4,71,61.9,88,196.9,89,6.6,7,2,0.0
4,75,1.0,0.0,0,166.7,113,148.3,122,186.9,121,10.1,3,3,0.0


In [ ]:
# APlico MLIB - Basado en RDD

In [ ]:
# Arboles de decision

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree

def labelData(data):
    # label: row[end], features: row[0:end-1]
    return data.map(lambda row: LabeledPoint(row[-1], row[:-1]))

training_data, testing_data = labelData(CV_data.rdd).randomSplit([0.8, 0.2])

model = DecisionTree.trainClassifier(training_data, numClasses=2, maxDepth=2,
                                     categoricalFeaturesInfo={1:2, 2:2},
                                     impurity='gini', maxBins=32)

print(model.toDebugString())

DecisionTreeModel classifier of depth 2 with 7 nodes
  If (feature 4 <= 264.4)
   If (feature 12 <= 3.5)
    Predict: 0.0
   Else (feature 12 > 3.5)
    Predict: 1.0
  Else (feature 4 > 264.4)
   If (feature 2 in {1.0})
    Predict: 0.0
   Else (feature 2 not in {1.0})
    Predict: 1.0



In [ ]:
print('Feature 12:', CV_data.columns[12])
print('Feature 4: ', CV_data.columns[4])
print('Feature 2: ', CV_data.columns[2])

Feature 12: Customer service calls
Feature 4:  Total day minutes
Feature 2:  Voice mail plan


In [ ]:
# Estructura
datapoint = training_data.take(1)[0]
print(datapoint.features) 
print(datapoint.label) 

[128.0,0.0,1.0,25.0,265.1,110.0,197.4,99.0,244.7,91.0,10.0,3.0,1.0]
0.0


In [ ]:
# Evaluamos el modelo

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

def getPredictionsLabels(model, test_data):
    predictions = model.predict(test_data.map(lambda r: r.features))
    return predictions.zip(test_data.map(lambda r: r.label))

def printMetrics(predictions_and_labels):
    metrics = MulticlassMetrics(predictions_and_labels)
    print('Precision of True '+ str(metrics.precision(1)))
    print('Precision of False'+ str(metrics.precision(0)))
    print('Recall of True    '+ str(metrics.recall(1)))
    print('Recall of False   '+ str(metrics.recall(0)))
    print('F-1 Score         '+ str(metrics.fMeasure(1.0)))
    print('Confusion Matrix\n'+ str(metrics.confusionMatrix().toArray()))

predictions_and_labels = getPredictionsLabels(model, testing_data)

printMetrics(predictions_and_labels)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Precision of True 0.6285714285714286
Precision of False0.9115913555992141
Recall of True    0.4943820224719101
Recall of False   0.9469387755102041
F-1 Score         0.5534591194968553
Confusion Matrix
[[464.  26.]
 [ 45.  44.]]
